In [1]:
import pandas as pd
import pickle
import numpy as np
with open('phone_train.pickle', 'rb') as fh1:
    train = pickle.load(fh1)
with open('phone_test1.pickle', 'rb') as fh2:
    test = pickle.load(fh2)
x_train=np.array(train[train.columns[:-1]])
y_train=np.array(train["label"])
x_test=np.array(test[train.columns[:-1]])
y_test=np.array(test["label"])

In [2]:
# Q1.1: Create my mypgc class
import math
class mypgc:
    def __init__(self):
        self.position=["Phoneonleft","Phoneonfront","Phoneonback","Phoneonbottom","Phoneontop","Phoneonright"]
        self.attribute=["mu","cov","prec","detcov","n","prior"]
        self.info={}
        self.wk={}
        self.wk0={}
        for p in self.position:
            self.info[p]={}
            for a in self.attribute:
                self.info[p][a]=None
    def fit(self, x, y):
        total=len(x)
        classes={}
#         index
        for i in self.position:
            classes[i]=np.where(y==i)[0]
#         x
        for i in self.position:
            classes[i]=x_train[classes[i],:]
#         mean, covariance, precision, det, n, prior    
        cov=0
        for i in self.position:
            self.info[i]["mu"]=np.mean(classes[i],axis=0)
            self.info[i]["cov"]=np.cov(classes[i],rowvar=False)
            self.info[i]["prec"]=np.linalg.pinv(self.info[i]["cov"])
            self.info[i]["detcov"]=np.linalg.det(self.info[i]["cov"])
            self.info[i]["n"]=len(classes[i])
            self.info[i]["prior"]=len(classes[i])/total
            cov+=self.info[i]["cov"]*self.info[i]["prior"]
        inv=np.linalg.pinv(cov)
#         wk, wk_0
#         ax=wk*x+wk_0
        for i in self.position:
            mu=self.info[i]["mu"]
            prior=self.info[i]["prior"]
            self.wk[i]=np.dot(inv, mu)
            self.wk0[i]=-0.5*np.dot(np.dot(mu.transpose(), cov), mu)+np.log(prior)
        self.checkinfo()
    def predict(self, x):
        ak=[]
        for i in self.position:
            ak.append(np.dot(x, self.wk[i]) + self.wk0[i])
        # nums
        result=np.argmax(ak, axis=0)
        ans=[]
        for i in result:
            ans.append(self.position[i])
        ans=np.array(ans)
        return ans
    def accuracy(self,ypred,y):
        return (np.sum(ypred==y)/len(y))
    def checkinfo(self):
        for p in self.position:
            print(p)
            for i in self.info[p]:
                if type(self.info[p][i])==type(np.array([])):
                    print("%s: \n"%i,str(self.info[p][i]))
                else:
                    print("%s: "%i,str(self.info[p][i]))
            print("-------------------------------------------------------------")

In [3]:
# Q1.2: train my model
pgc1 = mypgc()
pgc1.fit(x_train, y_train)

Phoneonleft
mu: 
 [ 9.92639446  0.09275717 -0.01933473]
n:  29522
cov: 
 [[ 0.00192839 -0.00619274  0.00063551]
 [-0.00619274  0.03200844 -0.00317611]
 [ 0.00063551 -0.00317611  0.00171996]]
detcov:  3.282281658720189e-08
prec: 
 [[1369.95113577  263.0134811   -20.49603676]
 [ 263.0134811    88.74589895   66.6992188 ]
 [ -20.49603676   66.6992188   712.14869845]]
prior:  0.17667582302494958
-------------------------------------------------------------
Phoneonfront
mu: 
 [ 0.11270401  0.14265129 -9.73579633]
n:  29079
cov: 
 [[ 0.00173617 -0.00562249  0.00071319]
 [-0.00562249  0.03069977 -0.00386164]
 [ 0.00071319 -0.00386164  0.00185582]]
detcov:  2.9712670166559285e-08
prec: 
 [[1415.5956848   258.4845593    -6.15206605]
 [ 258.4845593    91.32078358   90.68682325]
 [  -6.15206605   90.68682325  729.91124637]]
prior:  0.1740246683064328
-------------------------------------------------------------
Phoneonback
mu: 
 [ 0.20693328 -0.02615235  9.7963762 ]
n:  28566
cov: 
 [[ 0.00299006 

In [4]:
# Q1.3: List the first 20 predictions and well as the correct labels
#       Compute the accuracy for your model
ypred=pgc1.predict(x_test)
print("accuracy: ",pgc1.accuracy(ypred,y_test))
print("first 20: \n",ypred[0:20])
print("correct 20: \n",y_test[0:20])

accuracy:  0.7540922752691263
first 20: 
 ['Phoneonfront' 'Phoneonleft' 'Phoneonleft' 'Phoneonleft' 'Phoneonfront'
 'Phoneonright' 'Phoneonfront' 'Phoneonright' 'Phoneonfront'
 'Phoneonfront' 'Phoneonfront' 'Phoneonback' 'Phoneonleft' 'Phoneonback'
 'Phoneonleft' 'Phoneonback' 'Phoneonright' 'Phoneonright' 'Phoneonright'
 'Phoneonleft']
correct 20: 
 ['Phoneonfront' 'Phoneonbottom' 'Phoneonbottom' 'Phoneonbottom'
 'Phoneonfront' 'Phoneonright' 'Phoneonfront' 'Phoneontop' 'Phoneonfront'
 'Phoneonfront' 'Phoneonfront' 'Phoneonback' 'Phoneonleft' 'Phoneonback'
 'Phoneonbottom' 'Phoneonback' 'Phoneonright' 'Phoneonright' 'Phoneontop'
 'Phoneonleft']


In [5]:
# Q2.1
# 先看一下資料的大小和長相
import pandas as pd
import numpy as np
adult=pd.read_csv('adult.csv',header = None)
adult_test=pd.read_csv('adult_test.csv',header = None)
print("train:", adult.shape)
print("test:", adult_test.shape)
adult.head()

train: (32561, 15)
test: (16281, 15)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [6]:
# 看一下數值資料
adult.columns =adult_test.columns= ['age', 'workclass', 'fnlwgt', 'education', 'education_num',
'marital_status', 'occupation', 'relationship',
'race', 'sex', 'capital_gain', 'capital_loss',
'hours_per_week', 'native_country', 'income']
adult.describe()

,age,fnlwgt,education_num,capital_gain,capital_loss,hours_per_week
count,32561.000000,3.256100e+04,32561.000000,32561.000000,32561.000000,32561.000000
mean,38.581647,1.897784e+05,10.080679,1077.648844,87.303830,40.437456
std,13.640433,1.055500e+05,2.572720,7385.292085,402.960219,12.347429
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.178270e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.783560e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.370510e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000


In [7]:
# 是否有缺值
adult.info()
# adult_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
age               32561 non-null int64
workclass         32561 non-null object
fnlwgt            32561 non-null int64
education         32561 non-null object
education_num     32561 non-null int64
marital_status    32561 non-null object
occupation        32561 non-null object
relationship      32561 non-null object
race              32561 non-null object
sex               32561 non-null object
capital_gain      32561 non-null int64
capital_loss      32561 non-null int64
hours_per_week    32561 non-null int64
native_country    32561 non-null object
income            32561 non-null object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [8]:
# 沒有缺值是否有以其他符號代表缺值
notnum=[ 'workclass', 'education',
'marital_status', 'occupation', 'relationship',
'race', 'sex' , 'native_country', 'income']
for x in notnum:
    print(adult[x].unique())

[' State-gov' ' Self-emp-not-inc' ' Private' ' Federal-gov' ' Local-gov'
 ' ?' ' Self-emp-inc' ' Without-pay' ' Never-worked']
[' Bachelors' ' HS-grad' ' 11th' ' Masters' ' 9th' ' Some-college'
 ' Assoc-acdm' ' Assoc-voc' ' 7th-8th' ' Doctorate' ' Prof-school'
 ' 5th-6th' ' 10th' ' 1st-4th' ' Preschool' ' 12th']
[' Never-married' ' Married-civ-spouse' ' Divorced'
 ' Married-spouse-absent' ' Separated' ' Married-AF-spouse' ' Widowed']
[' Adm-clerical' ' Exec-managerial' ' Handlers-cleaners' ' Prof-specialty'
 ' Other-service' ' Sales' ' Craft-repair' ' Transport-moving'
 ' Farming-fishing' ' Machine-op-inspct' ' Tech-support' ' ?'
 ' Protective-serv' ' Armed-Forces' ' Priv-house-serv']
[' Not-in-family' ' Husband' ' Wife' ' Own-child' ' Unmarried'
 ' Other-relative']
[' White' ' Black' ' Asian-Pac-Islander' ' Amer-Indian-Eskimo' ' Other']
[' Male' ' Female']
[' United-States' ' Cuba' ' Jamaica' ' India' ' ?' ' Mexico' ' South'
 ' Puerto-Rico' ' Honduras' ' England' ' Canada' ' Germany' 

In [9]:
# 處理'？'
for x in notnum :
    print(x , ":" , sum(adult[x]==" ?"))
    print(x , ":" , sum(adult_test[x]==" ?"))
# 把問號換成缺值
adult=adult.replace(' ?',np.nan)
adult=adult.dropna()
adult.shape

workclass : 1836
workclass : 963
education : 0
education : 0
marital_status : 0
marital_status : 0
occupation : 1843
occupation : 966
relationship : 0
relationship : 0
race : 0
race : 0
sex : 0
sex : 0
native_country : 583
native_country : 274
income : 0
income : 0


(30162, 15)

In [10]:
# 處理label
def encode(df):
    x=pd.DataFrame.copy(df)
    from sklearn.preprocessing import OneHotEncoder, LabelEncoder
    x['income']=LabelEncoder().fit_transform(x['income'])
    x=pd.get_dummies(x)
    return x
adult_clean=encode(adult)
adult_test_clean=encode(adult_test)
adult_clean.head()

,age,fnlwgt,education_num,capital_gain,capital_loss,hours_per_week,income,workclass_ Federal-gov,workclass_ Local-gov,workclass_ Private,...,native_country_ Portugal,native_country_ Puerto-Rico,native_country_ Scotland,native_country_ South,native_country_ Taiwan,native_country_ Thailand,native_country_ Trinadad&Tobago,native_country_ United-States,native_country_ Vietnam,native_country_ Yugoslavia
0,39,77516,13,2174,0,40,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,50,83311,13,0,0,13,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,38,215646,9,0,0,40,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
3,53,234721,7,0,0,40,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
4,28,338409,13,0,0,40,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [11]:
# 把頻率小於10的拿掉
print(len(adult_clean.columns))
# for i in adult_clean.columns:
#     print(i, adult_clean[i].sum())
adult_clean=adult_clean.drop(columns=[i for i in adult_clean.columns if adult_clean[i].sum()<=10])
print(len(adult_test_clean.columns))
adult_test_clean=adult_test_clean[adult_clean.columns]
print(len(adult_test_clean.columns))
print(len(adult_clean.columns))

105
108
103
103


In [12]:
x_train=np.array(adult_clean.drop(columns=['income']))
y_train=np.array(adult_clean['income'])
x_test=np.array(adult_test_clean.drop(columns=['income']))
y_test=np.array(adult_test_clean['income'])

In [13]:
def sigmoid(z):
        z_p = 1/(1.0 + np.exp(-z))
        return z_p

In [14]:
# Q2.2 Derive the gradient and hession matrix for the new E(w)
dim=len(x_train[0])
lambda_vec=np.identity(dim)
b=np.sum(lambda_vec)/dim
w=np.dot(np.dot(np.linalg.pinv(np.dot(x_train.T,x_train)+b*np.identity(dim)),x_train.T),y_train)
y = sigmoid(np.dot(x_train, w))
R=np.identity(len(y))
np.fill_diagonal(R,y*(1-y))
H=np.dot(np.dot(x_train.T, R),x_train)
g=np.dot(x_train.T,y-y_train)
print("gradient: \n",g)
print("hession: \n",H)

gradient: 
 [ 3.29992293e+05  1.79383014e+09  8.66823367e+04 -6.47332098e+06
  2.27657005e+05  3.57651241e+05  1.95376493e+02  5.73308783e+02
  7.45788646e+03  8.01706243e+01  7.09610359e+02  3.79318231e+02
  7.08136404e+00  3.65535286e+02  4.79549143e+02  1.66661236e+02
  7.09889809e+01  1.34972017e+02  2.52202159e+02  2.08698887e+02
  3.10960225e+02  3.94191753e+02  9.13151096e+02 -2.64642053e+01
  3.70185697e+03  1.16412730e+02  2.24896625e+01 -3.99638736e+01
  2.33151025e+03  1.76676041e+03  2.91013386e+00  2.18916742e+03
  1.61648361e+02  4.50910850e+03  4.19810946e+02  3.53346543e+02
  1.48507213e+03  1.33129951e+03  5.27141317e+02  4.07948016e+02
  6.12492469e+02  7.98621729e+02  1.50638620e+03  7.07256125e+01
  6.44531124e+02  1.63576233e+02  1.05894194e+03  2.46170790e+02
  5.45895219e+02  1.93297427e+03  3.24341232e+03  4.18171318e+02
  2.18480012e+03  1.44573304e+03  1.77661247e+02  1.17388265e+02
  2.59788517e+02  1.13231669e+03  9.95765541e+01  7.79368229e+03
  4.05116048e

In [15]:
# Q2.3 Create your own mylogistic_l2 class
class mylogistic_l2:
    def __init__(self, reg_vec, max_iter = 1000, tol = 1e-5, add_intercept = True):
        self.reg_vec=reg_vec
        self.max_iter=max_iter
        self.tol=tol
        self.add_intercept=add_intercept
        self.w=None
    def error(self, yreal, yprob):
        e=-1*np.sum(np.dot(yreal, np.log(yprob+0.000001))+np.dot((1-yreal.T), np.log(1-yprob+0.000001)))
        l2=0.5*np.dot(np.dot(self.w.T, self.reg_vec), self.w)
        return e+l2
    def fit(self, x_train, y_train):
        if self.add_intercept:
            x_train=np.concatenate((x_train, np.ones((x_train.shape[0],1))), axis=1)
        else:
            self.reg_vec=np.delete(self.reg_vec, np.s_[-1], axis=0)
            self.reg_vec=np.delete(self.reg_vec, np.s_[-1], axis=1)
        dim=len(x_train[0])
        b=np.sum(self.reg_vec)/dim
        self.w=np.dot(np.dot(np.linalg.pinv(np.dot(x_train.T,x_train)+b*np.identity(dim)),x_train.T),y_train)
        y = sigmoid(np.dot(x_train, self.w))
        e=self.error(y_train, y)
        print(e)
        for i in range(self.max_iter):
            R=np.identity(len(y))
            np.fill_diagonal(R,y*(1-y))
            H=np.dot(np.dot(x_train.T, R),x_train)
            g=np.dot(x_train.T,y-y_train)
            w_new=self.w-np.dot(np.linalg.pinv(H),g)
            y = sigmoid(np.dot(x_train, w_new))
            e_new=self.error(y_train, y)
            print(e_new)
            if e-e_new<self.tol:
                break
            e=e_new
            self.w=w_new
#         print(self.w.shape)
#         print(self.w)
    def predict(self, x_test):
        if self.add_intercept:
            x_test=np.concatenate((x_test, np.ones((x_test.shape[0],1))), axis=1)
        return np.where(sigmoid(np.dot(x_test, self.w)) <= 0.5, 0, 1)
    def accuracy(self, ypred, y):
        return (np.sum(ypred==y)/len(y))

In [16]:
# case1
dim=len(x_train[0])
lambda_vec=np.identity(dim+1)
logic1 = mylogistic_l2(reg_vec = lambda_vec, max_iter = 1000, tol = 1e-5, add_intercept = True)
logic1.fit(x_train, y_train)
ypred = logic1.predict(x_test)
print("accuracy: ",logic1.accuracy(ypred,y_test))

21190.8477196014
11769.298599635138
10284.401720822736
9843.386109188164
9780.867862815181
9798.715494135937
accuracy:  0.8520975370063264


In [17]:
# case2
dim=len(x_train[0])
lambda_vec=np.identity(dim+1)
lambda_vec[dim][dim]=0
logic1 = mylogistic_l2(reg_vec = lambda_vec, max_iter = 1000, tol = 1e-5, add_intercept = True)
logic1.fit(x_train, y_train)
ypred = logic1.predict(x_test)
print("accuracy: ",logic1.accuracy(ypred,y_test))

21190.830635408507
11769.285554172016
10283.447456923994
9840.710327896384
9775.996487213495
9791.319279488109
accuracy:  0.8520975370063264


In [18]:
# case3
dim=len(x_train[0])
lambda_vec=np.identity(dim+1)*0.5
for i in range(6):
    lambda_vec[i][i]=1
lambda_vec[dim][dim]=0
logic1 = mylogistic_l2(reg_vec = lambda_vec, max_iter = 1000, tol = 1e-5, add_intercept = True)
logic1.fit(x_train, y_train)
ypred = logic1.predict(x_test)
print("accuracy: ",logic1.accuracy(ypred,y_test))

21190.593506284185
11769.228844454678
10281.120934834671
9834.419360517699
9763.773290645688
9768.976894341933
accuracy:  0.8519746944290891


In [19]:
# Q2.4
# from sklearn.cross_validation import train_test_split
from sklearn.model_selection import train_test_split
x_subtrain, x_tuning, y_subtrain, y_tuning = train_test_split(x_train, y_train, test_size=0.1, random_state=87)
print(len(x_subtrain))
print(len(y_subtrain))
print(len(x_tuning))
print(len(x_tuning))

27145
27145
3017
3017


In [20]:
# Choose a set of grids among a reasonable range. For example, 10 grids in [0.01, 100]
pick=[0.01*(10**(4/9))**i for i in range(10)]
temp=[]
for a in pick:
    lambda_vec=np.identity(dim+1)*a
    lambda_vec[dim][dim]=0
    logic1 = mylogistic_l2(reg_vec = lambda_vec, max_iter = 1000, tol = 1e-5, add_intercept = True)
    logic1.fit(x_subtrain, y_subtrain)
    ypred = logic1.predict(x_tuning)
    temp.append(logic1.accuracy(ypred,y_tuning))
    print("accuracy: ",logic1.accuracy(ypred,y_tuning))
a1=pick[np.argmax(temp, axis=0)]
print(a1)

19057.445680397548
10580.149256214989
9237.664368838357
8827.87089154236
8757.321152354963
8752.077470078128
8749.940400412475
8749.719963100048
8749.69553317932
8749.725726084904
accuracy:  0.8485250248591316
19057.45701955314
10580.15344876288
9237.750901878
8828.098296120292
8757.76424811106
8752.913397438471
8751.047281364094
8751.28099343024
accuracy:  0.8485250248591316
19057.488517008984
10580.165079680744
9237.991690479876
8828.731075039965
8758.997202217017
8755.239409740501
8754.127228407187
8755.624646623024
accuracy:  0.8485250248591316
19057.575513304026
10580.197162312601
9238.661730646925
8830.491814756797
8762.427950858806
8761.711645333971
8762.697252472812
accuracy:  0.8485250248591316
19057.815408010545
10580.28454437844
9240.526525968839
8835.391131035349
8771.974009997886
8779.72082836178
accuracy:  0.8488564799469672
19058.422167067787
10580.482894952638
9245.664712532993
8848.948631989613
8798.437140041831
8829.75043148594
accuracy:  0.8488564799469672
19060.1098

In [21]:
# Conduct grid search with the constraint that  a1=a2
temp=[]
for a in pick:
    lambda_vec=np.identity(dim+1)*a
    for i in range(6):
        lambda_vec[i][i]=a1
    lambda_vec[dim][dim]=0
    logic1 = mylogistic_l2(reg_vec = lambda_vec, max_iter = 1000, tol = 1e-5, add_intercept = True)
    logic1.fit(x_subtrain, y_subtrain)
    ypred = logic1.predict(x_tuning)
    temp.append(logic1.accuracy(ypred,y_tuning))
    print("accuracy: ",logic1.accuracy(ypred,y_tuning))
a2=pick[np.argmax(temp, axis=0)]
print(a2)

19057.460203037634
10580.149880198447
9237.66658884559
8827.875933136202
8757.330681666714
8752.096948078144
8749.967566002917
8749.756136965381
8749.734503131755
8749.764264267638
accuracy:  0.8485250248591316
19057.471089652405
10580.15404726593
9237.753055685975
8828.103190562302
8757.773496294274
8752.932285228353
8751.073622989938
8751.316072191586
accuracy:  0.8485250248591316
19057.501332494467
10580.165609560589
9237.993659823142
8828.735546693295
8759.005650624526
8755.256657493508
8754.15127823055
8755.656679327776
accuracy:  0.8485250248591316
19057.58480198553
10580.197514451977
9238.66318176898
8830.495102885407
8762.434162441881
8761.724327035174
8762.714929742117
accuracy:  0.8485250248591316
19057.815408010545
10580.28454437844
9240.526525968839
8835.391131035349
8771.974009997886
8779.72082836178
accuracy:  0.8488564799469672
19058.397267774642
10580.482362307814
9245.659612289954
8848.937944744437
8798.41771566347
8829.713490320466
accuracy:  0.8488564799469672
19060.

In [22]:
# Fix a1, and search a2
temp=[]
for a in pick:
    lambda_vec=np.identity(dim+1)*a2
    for i in range(6):
        lambda_vec[i][i]=a
    lambda_vec[dim][dim]=0
    logic1 = mylogistic_l2(reg_vec = lambda_vec, max_iter = 1000, tol = 1e-5, add_intercept = True)
    logic1.fit(x_subtrain, y_subtrain)
    ypred = logic1.predict(x_tuning)
    temp.append(logic1.accuracy(ypred,y_tuning))
    print("accuracy: ",logic1.accuracy(ypred,y_tuning))
a1=pick[np.argmax(temp, axis=0)]
print(a1)

19057.801301019346
10580.284125606659
9240.524270194448
8835.386081507118
8771.964481191684
8779.70136491659
accuracy:  0.8488564799469672
19057.801727548413
10580.284138105326
9240.524336025432
8835.386230164648
8771.964763836779
8779.701947891335
accuracy:  0.8488564799469672
19057.80291461289
10580.284173068601
9240.524522780188
8835.386649866563
8771.965558555172
8779.703584164532
accuracy:  0.8488564799469672
19057.806218921705
10580.284271581015
9240.525055771464
8835.387840301439
8771.967800679842
8779.70815515536
accuracy:  0.8488564799469672
19057.815408010545
10580.28454437844
9240.526525968839
8835.391131035349
8771.974009997886
8779.72082836178
accuracy:  0.8488564799469672
19057.840948007193
10580.285302126073
9240.530607153249
8835.400271955461
8771.99126656135
8779.7560988799
accuracy:  0.8488564799469672
19057.843432786904
10580.230367872728
9240.486466782048
8835.36382299
8771.96504450469
8779.821898741073
accuracy:  0.8488564799469672
19058.052080942158
10580.24680329

In [23]:
print("best a1 and a2: ",a1,a2)
dim=len(x_train[0])
lambda_vec=np.identity(dim+1)*a2
for i in range(6):
    lambda_vec[i][i]=a1
lambda_vec[dim][dim]=0
logic1 = mylogistic_l2(reg_vec = lambda_vec, max_iter = 1000, tol = 1e-5, add_intercept = True)
logic1.fit(x_train, y_train)
ypred = logic1.predict(x_test)
print("accuracy: ",logic1.accuracy(ypred,y_test))

best a1 and a2:  0.01 0.5994842503189409
21190.628962224135
11769.249888622582
10281.591909572153
9835.677670600162
9766.20843823305
9773.41145610041
accuracy:  0.8519746944290891


In [24]:
# Q2.5
from sklearn.linear_model import LogisticRegression
classifier=(LogisticRegression())
classifier.fit(x_train,y_train)
y_pred=classifier.predict(x_test)
print("accuracy: ",np.sum(ypred==y_test)/len(y_test))

accuracy:  0.8519746944290891


/Users/chessy/anaconda3/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
